In [1]:
import sys
sys.path.append("..")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,4"

from embedding_utils import *

########## Data setup ##########
PATH = "/homes/gws/hughchen/phase/upstream_embedding/"
hosp_data = 1
DATAPATH  = '{}data/min5_data/{}minimum5/hospital_{}/proc/'.format(PATH,"SAO2",hosp_data)
task = "nextfive"

########## Form Data #########
X_train_lst = []; X_valid_lst = []
y_train_lst = []; y_valid_lst = []

for i in range(0,len(X_lst)):
    feat = X_lst[i]
    
    data = load_train_val_data(DATAPATH,task,feat=feat,save_proc_y=True,
                               filter_zeros=False,impute_per_sample=False)
    X_train, y_train, X_valid, y_valid = data

    X_train_lst.append(X_train)
    X_valid_lst.append(X_valid)
    y_train_lst.append(y_train)
    y_valid_lst.append(y_valid)

Using TensorFlow backend.
W0929 17:26:25.804311 139970483787584 deprecation_wrapper.py:119] From ../embedding_utils.py:16: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0929 17:26:25.806766 139970483787584 deprecation_wrapper.py:119] From ../embedding_utils.py:16: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.

W0929 17:26:25.807962 139970483787584 deprecation_wrapper.py:119] From ../embedding_utils.py:17: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



[PROGRESS] Starting load_train_val_data()
[DEBUG] Loading from DATAPATH: /homes/gws/hughchen/phase/upstream_embedding/data/min5_data/SAO2minimum5/hospital_1/proc/
[DEBUG] Loading train_x_file: X_train_60_SAO2minimum5_featnum:2_featname:ECGRATE.npy
[DEBUG] Loading val_x_file  : X_val_60_SAO2minimum5_featnum:2_featname:ECGRATE.npy
[DEBUG] Loading train_y_file: y_train_60_ECGRATEnextfive.npy
[DEBUG] Loading val_y_file  : y_val_60_ECGRATEnextfive.npy
[DEBUG] Saving processed y files
[DEBUG] train_x.shape: (3785233, 60, 1), train_y.shape: (3785233, 5)
[DEBUG] val_x.shape  : (420582, 60, 1),   val_y.shape: (420582, 5)
[PROGRESS] Starting load_train_val_data()
[DEBUG] Loading from DATAPATH: /homes/gws/hughchen/phase/upstream_embedding/data/min5_data/SAO2minimum5/hospital_1/proc/
[DEBUG] Loading train_x_file: X_train_60_SAO2minimum5_featnum:3_featname:ETCO2.npy
[DEBUG] Loading val_x_file  : X_val_60_SAO2minimum5_featnum:3_featname:ETCO2.npy
[DEBUG] Loading train_y_file: y_train_60_ETCO2nextfiv

In [ ]:
import keras
from keras.utils import multi_gpu_model
from keras.layers import Input, LSTM, Dense, Dropout
from keras.models import Sequential, load_model, Model
from matplotlib import cm, pyplot as plt
from sklearn import metrics
from os.path import expanduser as eu
from os.path import isfile, join
import numpy as np
import random, time

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True,gpu_options = tf.GPUOptions(allow_growth=True))
set_session(tf.Session(config=config))
GPUNUM = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))

ISTUNE = False
opt_name="rmsprop";lr=0.001;drop=0.5
b_size=1000;epoch_num=200;nodesize=200
per_epoch_size = 300000

# Fixed hyperpara
print("[PROGRESS] Starting create_train_model()")
h1 = nodesize; h2 = nodesize
lookback = 60; loss_func = "mse"

# Set opt based on opt_name
if opt_name is "rmsprop":
    opt = keras.optimizers.RMSprop(lr)
elif opt_name is "sgd":
    opt = keras.optimizers.SGD(lr)
elif opt_name is "adam":
    opt = keras.optimizers.Adam(lr)

# Form the model name (for saving the model)
mod_name  = "multivariate_biglstmdropout_hd{}_{}task_{}n_{}n_{}ep".format(hosp_data,task,h1,h2,epoch_num)
mod_name += "_{}opt_{}lr".format(opt_name,lr)
mod_name += "_{}drop_{}bs_epochsize".format(drop,b_size,per_epoch_size)
if ISTUNE:
    MODDIR = PATH+"models/tune_biglstm/"+mod_name+"/"
else:
    MODDIR = PATH+"models/"+mod_name+"/"
    
loss_keys = ["loss", 'val_loss']
[loss_keys.append("val_dense_{}_loss".format(i)) for i in range(1,16)]

if not os.path.exists(MODDIR): 
    os.makedirs(MODDIR)
    with open(MODDIR+"loss.txt", "w") as f:
        f.write("\t".join(["i", "epoch_time"] + loss_keys)+"\n")

    ########## Form Model #########
    sig_lst     = []; encoded_lst = []
    for i in range(0,len(X_lst)):
        sig = Input(shape=(lookback,1))
        lstm1 = LSTM(h1, recurrent_dropout=drop, return_sequences=True)
        lstm2 = LSTM(h2, recurrent_dropout=drop, dropout=drop)

        encoded = lstm2(lstm1(sig))
        sig_lst.append(sig); encoded_lst.append(encoded)
    merged_vector = keras.layers.concatenate(encoded_lst, axis=-1)

    prediction_lst = []
    for i in range(0,len(X_lst)):
        prediction = Dense(5, activation='sigmoid')(merged_vector)
        prediction_lst.append(prediction)

    model = Model(inputs=sig_lst, outputs=prediction_lst)
    model = multi_gpu_model(model,gpus=GPUNUM)
    model.compile(optimizer=opt, loss=loss_func)
    
    ########## Training #########
    print("[PROGRESS] Starting train_model()")

    # Train and Save
    start_time = time.time()
    for i in range(0,epoch_num):
        train_subset_inds = np.random.choice(X_train_lst[0].shape[0],per_epoch_size,replace=False)
        X_train_lst_sub = [X[train_subset_inds] for X in X_train_lst]
        y_train_lst_sub = [y[train_subset_inds] for y in y_train_lst]
        history = model.fit(X_train_lst_sub, y_train_lst_sub, epochs=1, batch_size=b_size, 
                            validation_data=(X_valid_lst,y_valid_lst))

        # Save details about training
        epoch_time = time.time() - start_time
        write_lst = [i, epoch_time]
        write_lst += [history.history[k][0] for k in loss_keys]
        with open(MODDIR+"loss.txt", "a") as f:
            f.write("\t".join([str(round(e,5)) for e in write_lst])+"\n")

        # Save model each iteration
        val_loss = history.history['val_loss'][0]
        model.save("{}val_loss:{}_epoch:{}_{}.h5".format(MODDIR,val_loss,i,mod_name))

# # Return the best validation performance (if hyperpara tuning)
# min_mod_name = [f for f in sorted(os.listdir(MODDIR)) if "val_loss" in f][0]
# best_val = float(min_mod_name.split("val_loss:")[1].split("_")[0])
# return(best_val)

W0929 17:29:44.774096 139970483787584 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0929 17:29:44.775962 139970483787584 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



[PROGRESS] Starting create_train_model()


W0929 17:29:45.484375 139970483787584 deprecation.py:506] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0929 17:30:19.231067 139970483787584 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



[PROGRESS] Starting train_model()


W0929 17:30:26.136471 139970483787584 deprecation.py:323] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 300000 samples, validate on 420582 samples
Epoch 1/1
300000/300000 [==============================] - 2033s 7ms/step - loss: 11.8529 - dense_1_loss: 0.6943 - dense_2_loss: 0.7002 - dense_3_loss: 0.8320 - dense_4_loss: 0.8320 - dense_5_loss: 0.7042 - dense_6_loss: 0.9597 - dense_7_loss: 0.9579 - dense_8_loss: 0.9560 - dense_9_loss: 0.7797 - dense_10_loss: 0.7329 - dense_11_loss: 0.7796 - dense_12_loss: 0.6995 - dense_13_loss: 0.9819 - dense_14_loss: 0.7482 - dense_15_loss: 0.4950 - val_loss: 11.3902 - val_dense_1_loss: 0.7074 - val_dense_2_loss: 0.6846 - val_dense_3_loss: 0.7733 - val_dense_4_loss: 0.7734 - val_dense_5_loss: 0.6137 - val_dense_6_loss: 0.9718 - val_dense_7_loss: 0.9452 - val_dense_8_loss: 0.9622 - val_dense_9_loss: 0.7839 - val_dense_10_loss: 0.6639 - val_dense_11_loss: 0.7839 - val_dense_12_loss: 0.7020 - val_dense_13_loss: 0.8791 - val_dense_14_loss: 0.6986 - val_dense_15_loss: 0.4472
Train on 300000 samples, validate on 420582 samples
Epoch 1/1
300000/300000 

In [ ]:
start_time = time.time()
for i in range(50,200):
    train_subset_inds = np.random.choice(X_train_lst[0].shape[0],per_epoch_size,replace=False)
    X_train_lst_sub = [X[train_subset_inds] for X in X_train_lst]
    y_train_lst_sub = [y[train_subset_inds] for y in y_train_lst]
    history = model.fit(X_train_lst_sub, y_train_lst_sub, epochs=1, batch_size=b_size, 
                        validation_data=(X_valid_lst,y_valid_lst))

    # Save details about training
    epoch_time = time.time() - start_time
    write_lst = [i, epoch_time]
    write_lst += [history.history[k][0] for k in loss_keys]
    with open(MODDIR+"loss.txt", "a") as f:
        f.write("\t".join([str(round(e,5)) for e in write_lst])+"\n")

    # Save model each iteration
    val_loss = history.history['val_loss'][0]
    model.save("{}val_loss:{}_epoch:{}_{}.h5".format(MODDIR,val_loss,i,mod_name))

### Is the difference in loss due to the imputation?

### Compare the y labels to the upstream embedding ones

# Old code below here

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,4"

import numpy as np
from tbi_downstream_prediction import split_data

PATH = "/homes/gws/hughchen/phase/tbi_subset/"
DPATH = PATH+"tbi/processed_data/hypoxemia/"
data_type = "raw[top11]"

feat_lst = ["ECGRATE", "ETCO2", "ETSEV", "ETSEVO", "FIO2", "NIBPD", "NIBPM", 
            "NIBPS","PEAK", "PEEP", "PIP", "RESPRATE", "SAO2", "TEMP1", "TV"]

# Exclude these features
weird_feat_lst = ["ETSEV", "PIP", "PEEP", "TV"]
feat_inds = np.array([feat_lst.index(feat) for feat in feat_lst if feat not in weird_feat_lst])
feat_lst2 = [feat for feat in feat_lst if feat not in weird_feat_lst]

y_tbi = np.load(DPATH+"tbiy.npy")
X_tbi = np.load(DPATH+"X_tbi_imp_standard.npy")

X_tbi2 = X_tbi[:,feat_inds,:]
(X_test, y_test, X_valid, y_valid, X_train, y_train) = split_data(DPATH,X_tbi2,y_tbi,flatten=False)

PATH = "/homes/gws/hughchen/phase/tbi_subset/"
RESULTPATH = PATH+"results/"
label_type = "desat_bool92_5_nodesat"
lstm_type = "biglstmdropoutv3_{}".format(label_type)
RESDIR = '{}{}/'.format(RESULTPATH, lstm_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)

import keras
from keras.utils import multi_gpu_model
from keras.layers import Input, LSTM, Dense, Dropout
from keras.models import Sequential, load_model, Model
from matplotlib import cm, pyplot as plt
from sklearn import metrics
from os.path import expanduser as eu
from os.path import isfile, join
import numpy as np
import random, time

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True,gpu_options = tf.GPUOptions(allow_growth=True))
set_session(tf.Session(config=config))
GPUNUM = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))

def load_min_model_helper(MPATH):
    print("[PROGRESS] Starting load_min_model_helper()")
    print("[DEBUG] MPATH {}".format(MPATH))
    mfiles = os.listdir(MPATH)
    full_mod_name = MPATH.split("/")[-1]
    mfiles = [f for f in mfiles if "val_loss" in f]
    loss_lst = [float(f.split("val_loss:")[1].split("_")[0]) for f in mfiles]
    min_ind = loss_lst.index(min(loss_lst))
    min_mod_name = "{}/{}".format(MPATH,mfiles[min_ind])
    print("[DEBUG] min_mod_name {}".format(mfiles[min_ind]))
    return(load_model(min_mod_name))

def create_train_model(opt_name="rmsprop",lr=0.001,drop=0.5,
                       b_size=1000,epoch_num=50,is_tune=False,
                       nodesize=200):
    # Fixed hyperpara
    print("[PROGRESS] Starting create_train_model()")
    h1 = nodesize
    h2 = nodesize
    lookback = 60
    loss_func = "binary_crossentropy"

    # Set opt based on opt_name
    if opt_name is "rmsprop":
        opt = keras.optimizers.RMSprop(lr)
    elif opt_name is "sgd":
        opt = keras.optimizers.SGD(lr)
    elif opt_name is "adam":
        opt = keras.optimizers.Adam(lr)

    # Form the model name (for saving the model)
    mod_name  = "multivariate_biglstmdropoutv3_{}n_{}n_{}ep".format(h1,h2,epoch_num)
    mod_name += "_{}opt_{}lr".format(opt_name,lr)
    mod_name += "_{}drop_{}bs".format(drop,b_size)
    if ISTUNE:
        MODDIR = PATH+"models/tune_biglstm/"+mod_name+"/"
    else:
        MODDIR = PATH+"models/"+mod_name+"/"
    if is_tune: MODDIR = PATH+"models/tune_biglstm/"+mod_name+"/"
    if not os.path.exists(MODDIR): 
        os.makedirs(MODDIR)
        with open(MODDIR+"loss.txt", "w") as f:
            f.write("%s\t%s\t%s\t%s\n" % ("i", "train_loss", "val_loss", "epoch_time"))

        ########## Form Model/Data #########
        X_train_lst = []; X_valid_lst = []; X_test_lst  = []
        sig_lst     = []; encoded_lst = []

        for i in range(0,len(feat_lst2)):
            X_train_lst.append(X_train[:,:,i:(i+1)])
            X_valid_lst.append(X_valid[:,:,i:(i+1)])
            X_test_lst.append(X_test[:,:,i:(i+1)])

            sig = Input(shape=(lookback,1))
            lstm1 = LSTM(h1, recurrent_dropout=drop, return_sequences=True)
            lstm2 = LSTM(h2, recurrent_dropout=drop, dropout=drop)

            encoded = lstm2(lstm1(sig))
            sig_lst.append(sig); encoded_lst.append(encoded)

        merged_vector = keras.layers.concatenate(encoded_lst, axis=-1)
        predictions = Dense(1, activation='sigmoid')(merged_vector)

        model = Model(inputs=sig_lst, outputs=predictions)
        model = multi_gpu_model(model, gpus=GPUNUM)
        model.compile(optimizer=opt, loss=loss_func)

        ########## Training #########
        print("[PROGRESS] Starting train_model()")

        # Train and Save
        start_time = time.time()
        for i in range(0,epoch_num):
            history = model.fit(X_train_lst, y_train, epochs=1, batch_size=b_size, 
                                validation_data=(X_valid_lst,y_valid))

            # Save details about training
            train_loss = history.history['loss'][0]
            val_loss = history.history['val_loss'][0]
            epoch_time = time.time() - start_time
            with open(MODDIR+"loss.txt", "a") as f:
                f.write("%d\t%f\t%f\t%f\n" % (i, train_loss, val_loss, epoch_time))

            # Save model each iteration
            model.save("{}val_loss:{}_epoch:{}_{}.h5".format(MODDIR,val_loss,i,mod_name))
        
    # Return the best validation performance (if hyperpara tuning)
    min_mod_name = [f for f in sorted(os.listdir(MODDIR)) if "val_loss" in f][0]
    best_val = float(min_mod_name.split("val_loss:")[1].split("_")[0])
    return(best_val)

def eval_test_model(opt_name="rmsprop",lr=0.001,drop=0.5,
                    b_size=1000,epoch_num=50,nodesize=200):    
    
    # Fixed hyperpara
    print("[PROGRESS] Starting create_train_model()")
    lookback = 60; h1 = 200; h2 = 200
    loss_func = "binary_crossentropy"

    # Form the model name (for saving the model)
    mod_name  = "multivariate_biglstmdropoutv3_{}n_{}n_{}ep".format(h1,h2,epoch_num)
    mod_name += "_{}opt_{}lr".format(opt_name,lr)
    mod_name += "_{}drop_{}bs".format(drop,b_size)
    MODDIR = PATH+"models/"+mod_name+"/"
    
    ########## Form Data #########
    X_test_lst  = []

    for i in range(0,len(feat_lst2)):
        X_test_lst.append(X_test[:,:,i:(i+1)])
    
    ########## Testing (optional) #########
    model = load_min_model_helper(MODDIR)
    model = multi_gpu_model(model,gpus=GPUNUM)
    hosp_data = "tbi"
    save_path = RESDIR+"hosp{}_data/{}/".format(hosp_data,data_type)
    if not os.path.exists(save_path): os.makedirs(save_path)
    ypred = model.predict(X_test_lst)
    np.save(save_path+"ypred.npy",ypred)
    np.save(save_path+"y_test.npy",y_test)
    auc = metrics.average_precision_score(y_test, ypred)
    np.random.seed(231)
    auc_lst = []
    roc_auc_lst = []
    
    # Randomly sample the test set
    for i in range(0,100):
        inds = np.random.choice(X_test.shape[0], X_test.shape[0], replace=True)
        auc = metrics.average_precision_score(y_test[inds], ypred[inds])
        auc_lst.append(auc)
        roc_auc = metrics.roc_auc_score(y_test[inds], ypred[inds])
        roc_auc_lst.append(roc_auc)
    auc_lst = np.array(auc_lst)
    roc_auc_lst = np.array(roc_auc_lst)
    print("[DEBUG] auc_lst.mean(): {}".format(auc_lst.mean()))
    print("[DEBUG] roc_auc_lst.mean(): {}".format(roc_auc_lst.mean()))

    SP = RESDIR+"hosp{}_data/".format(hosp_data)
    f = open('{}conf_int_hospdata{}_prauc.txt'.format(SP,hosp_data),'a')
    f.write("{}, {}+-{}\n".format(data_type,auc_lst.mean().round(4),2*np.std(auc_lst).round(4)))
    f.close()
    f = open('{}conf_int_hospdata{}_rocauc.txt'.format(SP,hosp_data),'a')
    f.write("{}, {}+-{}\n".format(data_type,roc_auc_lst.mean().round(4),2*np.std(roc_auc_lst).round(4)))
    f.close()
    np.save("{}auc_lst".format(save_path,data_type), auc_lst)
    np.save("{}roc_auc_lst".format(save_path,data_type), roc_auc_lst)

## Hyperparameter tuning

In [ ]:
# # Set of hyperparameters to tune over
# node_lst   = [50, 100, 200]
# opt_lst    = ["rmsprop", "adam", "sgd"]
# lr_lst     = [0.01, 0.001, 0.0001]
# drop_lst   = [0.3, 0.5, 0.7]
# ISTUNE = True

# # Greedily find best opt
# min_loss_lst = [
#     create_train_model(opt_name="rmsprop",lr=0.001,drop=0.5,
#                        b_size=1000,nodesize=para)
#     for para in node_lst
# ]
# best_nodesize = node_lst[min_loss_lst.index(min(min_loss_lst))]
# print("best best_nodesize: {}".format(best_nodesize))

# # Greedily find best opt
# min_loss_lst = [
#     create_train_model(opt_name=para,lr=0.001,drop=0.5,
#                        b_size=1000,nodesize=best_nodesize)
#     for para in opt_lst
# ]
# best_optname = opt_lst[min_loss_lst.index(min(min_loss_lst))]
# print("best optname: {}".format(best_optname))

# # Greedily find best lr
# min_loss_lst = [
#     create_train_model(opt_name=best_optname,lr=para,drop=0.5,
#                        b_size=1000,nodesize=best_nodesize)
#     for para in lr_lst
# ]
# best_lr = lr_lst[min_loss_lst.index(min(min_loss_lst))]
# print("best lr: {}".format(best_lr))

# # Greedily find best dropout
# min_loss_lst = [
#     create_train_model(opt_name=best_optname,lr=best_lr,drop=para,
#                        b_size=1000,nodesize=best_nodesize)
#     for para in drop_lst
# ]
# best_drop = drop_lst[min_loss_lst.index(min(min_loss_lst))]
# print("best drop: {}".format(best_drop))

In [ ]:
create_train_model(opt_name=best_optname,lr=best_lr,drop=best_drop,
                   b_size=1000,nodesize=best_nodesize,epoch_num=200,
                   is_tune=False)
eval_test_model(opt_name=best_optname,lr=best_lr,drop=best_drop,
                b_size=1000,nodesize=best_nodesize,epoch_num=200)

In [ ]:
eval_test_model(opt_name=best_optname,lr=best_lr,drop=best_drop,
                b_size=1000,nodesize=best_nodesize,epoch_num=200)